In [ ]:
!pip install transformers accelerate einops sentencepiece
!pip install torch --upgrade
!pip install flask flask-cors spacy
!python -m spacy download en_core_web_sm
!pip install transformers torch accelerate bitsandbytes
!pip install -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 109.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
#get access tokens from huggingface website
from huggingface_hub import login
login("") #add your huggingface access token here


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

#Load Mistral-7B
model_name = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)
save_path = "/content/mistral-7b"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("Model saved at:", save_path)


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:4086: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(


Saving checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Model saved at: /content/mistral-7b


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
save_path = "/content/drive/MyDrive/mistral-7b"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("Model saved in Google Drive at:", save_path)

Saving checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Model saved in Google Drive at: /content/drive/MyDrive/mistral-7b


# Load the save model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. Define the path to your downloaded model files
#    (The folder containing config.json, .safetensors files, etc.)
model_path = "/content/drive/MyDrive/mistral-7b" # Or "path/to/your/drive/mistral-7b"

print("--- Loading model and tokenizer from local path ---")

# 2. Load the tokenizer and model
#    device_map="auto" will automatically use the GPU (cuda) if available.
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16, # Use bfloat16 for better performance
    device_map="auto"
)

print("--- Model and tokenizer loaded successfully! ---")

--- Loading model and tokenizer from local path ---


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

--- Model and tokenizer loaded successfully! ---


# Depression

**Minimal**

In [ ]:
prompt = """<s>[INST]
You are a gentle mental wellness assistant.
Generate exactly 3 short, simple self-care tasks for someone experiencing Minimal Depression.
Minimal Depression means the person feels slightly low, but can carry out daily activities.
Each task should be very easy, take only a few minutes, and help gently lift mood.
Format as a bullet list, one task per bullet.
Do not combine multiple actions in one bullet or add commentary.
[/INST]"""


# --- Tokenize Input ---
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# --- Generate Output ---
model.eval()
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=False,               # Greedy decoding (deterministic)
        repetition_penalty=1.2,        # Prevents repetition
        pad_token_id=tokenizer.eos_token_id
    )

# --- Decode Response ---
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# --- Post-process: Keep Only Assistant Response ---
if "[/INST]" in response:
    response = response.split("[/INST]")[-1].strip()

print("\n--- Model Response ---")
print(response)



--- Model Response ---
1. Take a walk outside.
2. Listen to your favorite music.
3. Read something funny online.


**Mild**

In [ ]:
prompt = """<s>[INST]
You are a mental assistant.
The person has Mild Depression, meaning they feel down more often and have reduced energy or motivation.
List 3 simple, practical ways to engage with their surroundings or do something creative today to lift their mood.
Each task should be achievable and specific.
Present as a bullet list with one step per bullet.
[/INST]"""



# --- Tokenize Input ---
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# --- Generate Output ---
model.eval()
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=False,               # Greedy decoding (deterministic)
        repetition_penalty=1.2,        # Prevents repetition
        pad_token_id=tokenizer.eos_token_id
    )

# --- Decode Response ---
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# --- Post-process: Keep Only Assistant Response ---
if "[/INST]" in response:
    response = response.split("[/INST]")[-1].strip()

print("\n--- Model Response ---")
print(response)



--- Model Response ---
1) Go for a walk outside and take in your surroundings.
2) Call up an old friend you haven't spoken to in a while.
3) Try making some art - it doesn't have to be good!


**Moderate**

In [ ]:
prompt = """<s>[INST]
You are an understanding mental health coach.
The individual is living with moderate depression — they can manage basic responsibilities but often feel emotionally heavy, tired, and disconnected, making it difficult to stay consistent with daily routines.
Suggest 3 realistic tasks that require gentle effort yet are still doable, with the intention of lifting mood or building a small sense of progress.
Each task must be action-driven, clearly defined, and measurable.
Format the response as bullet points, one per line, with no added commentary.
[/INST]"""


# --- Tokenize Input ---
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# --- Generate Output ---
model.eval()
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=False,               # Greedy decoding (deterministic)
        repetition_penalty=1.2,        # Prevents repetition
        pad_token_id=tokenizer.eos_token_id
    )

# --- Decode Response ---
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# --- Post-process: Keep Only Assistant Response ---
if "[/INST]" in response:
    response = response.split("[/INST]")[-1].strip()

print("\n--- Model Response ---")
print(response)


--- Model Response ---
1. Take a walk outside for at least 20 minutes each day. This will help you get some fresh air and exercise, which can improve your mood and energy levels.
2. Practice mindfulness meditation for 5-10 minutes every morning before starting your day. Mindfulness meditation has been shown to reduce stress and anxiety, which can contribute to feelings of depression.
3. Make time for self-care activities that bring you joy, such as reading a book,


**Severe**

In [ ]:
prompt = """<s>[INST]
You are a compassionate mental health coach.
The individual is facing severe depression — even small tasks feel overwhelming, energy is extremely low, and daily functioning is often disrupted.
Offer 3 very low-effort actions that can be done even on the hardest days, focused on providing relief or a small sense of stability.
Each action must be simple, clearly defined, and measurable.
Display them as bullet points, one per line, without any additional commentary.
[/INST]"""



# --- Tokenize Input ---
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# --- Generate Output ---
model.eval()
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=False,               # Greedy decoding (deterministic)
        repetition_penalty=1.2,        # Prevents repetition
        pad_token_id=tokenizer.eos_token_id
    )

# --- Decode Response ---
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# --- Post-process: Keep Only Assistant Response ---
if "[/INST]" in response:
    response = response.split("[/INST]")[-1].strip()

print("\n--- Model Response ---")
print(response)


--- Model Response ---
1. Take a shower.
2. Eat something healthy (fruit, veggies).
3. Go for a walk outside.


# Anxiety

**Minimal**

In [ ]:
prompt = """<s>[INST]
You are a mental wellness assistant.
Generate exactly 3 short, actionable self-care tasks for a person experiencing Minimal Anxiety.
Each task should be simple and doable within a day.
Format the response as a bullet list with no extra commentary.
[/INST]"""

# --- Tokenize Input ---
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# --- Generate Output ---
model.eval()
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=False,               # Greedy decoding (deterministic)
        repetition_penalty=1.2,        # Prevents repetition
        pad_token_id=tokenizer.eos_token_id
    )

# --- Decode Response ---
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# --- Post-process: Keep Only Assistant Response ---
if "[/INST]" in response:
    response = response.split("[/INST]")[-1].strip()

print("\n--- Model Response ---")
print(response)


--- Model Response ---
## Answer:

1. Take deep breaths to calm your mind and body.
2. Engage in activities that bring you joy or relaxation such as reading, listening to music, or taking a walk outside.
3. Practice positive affirmations by repeating phrases like "I am capable" or "I can handle this."


**Mild**

In [ ]:
prompt = """<s>[INST]
You are a mental wellness assistant specialized in anxiety support.
Generate **exactly 3 very short, actionable self-care tasks** for a person experiencing **Mild Anxiety**.
Mild Anxiety means the person feels slightly uneasy, a little tense, but can carry out most daily activities without difficulty.
Each task must be:
- One sentence only.
- Simple, light, and calming.
- Achievable within one day.
- Different from the tasks for Minimal, Moderate, or Severe Anxiety.
Output must be:
- Exactly 3 bullet points.
- Strictly one task per bullet.
- No explanations, examples, dialogue, or extra text.
- No repeated or combined tasks.
[/INST]"""

# --- Tokenize Input ---
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# --- Generate Output ---
model.eval()
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=False,               # Greedy decoding (deterministic)
        repetition_penalty=1.2,        # Prevents repetition
        pad_token_id=tokenizer.eos_token_id
    )

# --- Decode Response ---
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# --- Post-process: Keep Only Assistant Response ---
if "[/INST]" in response:
    response = response.split("[/INST]")[-1].strip()

print("\n--- Model Response ---")
print(response)



--- Model Response ---
# MILD ANXIETY TASKS

1. Take a walk outside to get some fresh air.
2. Listen to soothing music while doing something you enjoy (e.g., reading, drawing).
3. Practice deep breathing exercises to calm your mind and body.


**Moderate**

In [ ]:
prompt = """<s>[INST]
You are a compassionate mental wellness assistant.
Generate **exactly 3 short, concise, actionable self-care tasks** for a person experiencing **Moderate Anxiety**.
Moderate Anxiety means the person feels noticeably tense, restless, or worried, and may struggle to focus on daily tasks.
Each task should be **simple, easy to do, and no longer than one sentence**.
Tasks should be slightly more engaging than Minimal or Mild Anxiety, but still achievable within one day.
Format the response as a **bullet list**, with one task per bullet.
Do not repeat tasks or combine multiple actions in one bullet.
Do not add extra commentary.
[/INST]"""

# --- Tokenize Input ---
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# --- Generate Output ---
model.eval()
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=False,               # Greedy decoding (deterministic)
        repetition_penalty=1.2,        # Prevents repetition
        pad_token_id=tokenizer.eos_token_id
    )

# --- Decode Response ---
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# --- Post-process: Keep Only Assistant Response ---
if "[/INST]" in response:
    response = response.split("[/INST]")[-1].strip()

print("\n--- Model Response ---")
print(response)



--- Model Response ---
## Examples:

* Take a walk outside
* Listen to calming music
* Practice deep breathing exercises

---

# Moderate Anxiety Tasks

1. Go for a walk outside
2. Listen to calming music
3. Practice deep breathing exercises
4. Engage in mindfulness meditation
5. Write down your thoughts and feelings in a journal
6. Spend time with loved ones who make you feel supported
7


**Severe**

In [ ]:
prompt = """<s>[INST]
You are a mental wellness coach for individuals experiencing intense anxiety.
Generate exactly 3 **short, clear, and actionable self-care tasks** for someone with **Severe Anxiety**.
Severe Anxiety means the person feels very tense, restless, or panicky, and may find daily activities overwhelming.
Each task must be:
- One sentence only.
- Gentle, grounding, and achievable within one day.
- Different from tasks for Minimal, Mild, or Moderate Anxiety.
Output format:
- Exactly 3 bullet points.
- One task per bullet.
- No explanations, dialogue, or extra text.
[/INST]"""

# --- Tokenize Input ---
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# --- Generate Output ---
model.eval()
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=False,          # Greedy decoding
        repetition_penalty=1.2,   # Prevents repetition
        pad_token_id=tokenizer.eos_token_id
    )

# --- Decode Response ---
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# --- Post-process: Keep Only Assistant Response ---
if "[/INST]" in response:
    response = response.split("[/INST]")[-1].strip()

print("\n--- Model Response ---")
print(response)



--- Model Response ---
## Tasks

1. Take deep breaths in through your nose and out through your mouth to calm yourself down.
2. Go outside and take a walk around the block to get some fresh air and exercise.
3. Write down all of your worries on paper so you can see them clearly and then focus on something else instead.
4. Listen to calming music or nature sounds to help relax your mind and body.
5. Talk about how you're
